In [1]:
import os

import cv2

import fitz
from tqdm import tqdm

In [3]:
%load_ext watermark
%watermark -v -m -p fitz,cv2

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.5
IPython version      : 8.7.0

fitz: ('1.21.0', '1.21.0', '20221108000001')
cv2 : 4.6.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 69 Stepping 1, GenuineIntel
CPU cores   : 4
Architecture: 64bit



In [4]:
workdir = 'pdf_book'
pdf_name = 'Бланшар. Одноминутный менеджер и ситуационное руководство.pdf'

In [5]:
def add_page(doc, png_name, factor=None):
    img = fitz.open(png_name)  # open pic as document
    rect = img[0].rect  # pic dimension
    if factor: rect /= factor
    pdfbytes = img.convert_to_pdf()  # make a PDF stream
    img.close()  # no longer needed
    imgPDF = fitz.open("pdf", pdfbytes)  # open stream as PDF
    page = doc.new_page(width  = rect.width,  # new page with ...
                        height = rect.height)  # pic dimension
    page.show_pdf_page(rect, imgPDF, 0)  # image fills the page
    imgPDF.close()

In [6]:
doc_out = fitz.open()

doc_in = fitz.Document(os.path.join(workdir, pdf_name))
for i in tqdm(range(len(doc_in))):
    for img in doc_in.get_page_images(i):
        xref = img[0]
        pix = fitz.Pixmap(doc_in, xref)
        pix.save(os.path.join(workdir, 'lr.png'))

        lr = cv2.imread(os.path.join(workdir, 'lr.png'))
        l = lr[:,:lr.shape[1]//2, :]
        r = lr[:,lr.shape[1]//2:, :]
        cv2.imwrite(os.path.join(workdir, 'l.png'), l)
        cv2.imwrite(os.path.join(workdir, 'r.png'), r)

        add_page(doc_out, os.path.join(workdir, 'l.png'), factor=3)
        add_page(doc_out, os.path.join(workdir, 'r.png'), factor=3)

doc_out.save(os.path.join(workdir, pdf_name.replace('.pdf', '_splitted.pdf')))

print("Done!")

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:30<00:00,  2.34it/s]

Done!
